In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285397 sha256=a67e2d4bd8bb877c86eed459e7d550b5cb36240a77d1d888c2d426301aa09da2
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [15]:
import pandas as pd

In [22]:
df = pd.read_excel("/ARE YOU A CAKE OTAKU.xlsx")

Column cleaning/ Reanaming






In [23]:
new_column_names = {
    'If yes. What is your favorite Flavor?': 'Favorite Flavor',
    'If YES,What other activities do enjoy besides Cake 🎂?': 'Other activities enjoyed',
    'Would you please describe to us your cake experience ?': 'Cake experience',
    'Why do you buy cake?':'Reason to buy Cake',
    'How often do you buy cake?':'Frequency of buying cake',
    'Where do you purchase your cakes from? And why.':'Place of Purchase',
    'How do you order your cakes?':'Medium of order',
    'What problems / issues have you faced when ordering cake?':'Problem faced ordering cake',
    'What do you wish Bakers could do differently?':'Advise to bakers',
    'Where do you live?':'Residence',
    'How old are you?':'Age',
    'What is the perfect size of cake to you?':'Perfect size of cake',
    'Do you love cake?':'Love Cake'

    # Add more mappings as needed
}
df.rename(columns=new_column_names, inplace=True)

drop columns

In [24]:
columns_to_drop = [
    'Timestamp',
    'What emotions do you experience when buying or eating cake?',
    'Which of these mediums do you use and trust?',
    '@dropdown',
    'If No. Why not?'


]
df.drop(columns=columns_to_drop, inplace=True)

clean white space

In [25]:
DF = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

clean place of purchase column

In [28]:
pip install fuzzywuzzy

In [29]:
import pandas as pd
from fuzzywuzzy import process  # Import fuzzy string matching
import re  # Import regular expressions library

# Load your data (replace this with your actual data loading code)
# df = pd.read_csv('your_data.csv')

# Clean and categorize function
def clean_and_categorize(response):
    if isinstance(response, str):  # Check if the response is a string
        cleaned_response = response.lower().strip().replace(".", "").replace(",", "")

        categories = {
            'Bakeries': ['baker', 'bakery'],
            'Cafe Javas': [ 'cafe', 'javas'],
            'Piato': ['piato'],
            'serena': ['Serena'],
            'Supermarkets': ['supermarket'],
            'Online': ['online'],
            'Vendor': ['Vendor'],
            'Friends/Family': ['friend', 'wife'],
            'Other': []  # Default category
        }

        # Split cleaned response into words
        words = re.findall(r'\w+', cleaned_response)

        for category, keywords in categories.items():
            for keyword in keywords:
                if keyword in words:
                    return category
                # Fuzzy string matching
                matches = process.extract(keyword, words, limit=1)
                if matches[0][1] >= 80:  # Adjust similarity threshold as needed
                    return category

        return 'Other'  # If no category matches
    else:
        return 'Other'  # Handle non-string values

# Apply the clean_and_categorize function to the 'Place of Purchase' column
DF['category'] = DF['Place of Purchase'].apply(clean_and_categorize)

# Display the cleaned and categorized DataFrame
print(DF)

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


   Love Cake                                    Favorite Flavor  \
0        Yes                                    Chocolate fudge   
1        Yes                     Everything minus plain vanilla   
2        Yes                                    Chocolate fudge   
3        Yes                                          Chocolate   
4         No                                                Non   
5        Yes                                             Banana   
6        Yes                                         Red velvet   
7        Yes                                         Red velvet   
8        Yes                                          Blueberry   
9        Yes                                             Banana   
10       Yes                                        Dark forest   
11       Yes                                               Mint   
12       Yes                                       Black Forest   
13       Yes                                            Vanill

Anayse with Pyspark

In [2]:
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark import SparkContext

In [3]:
sc =SparkContext()

In [4]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[1]").appName('Spark').getOrCreate()

In [8]:
survey = spark.read.csv("ARE YOU A CAKE OTAKU_Cleaned1.csv", header=True, inferSchema=True)

In [9]:
survey.createOrReplaceTempView("table1")
survey.show(10)

+---------+--------------------+------------------------+--------------------+--------------------+--------------------+------------------------+---------------------+--------------------+--------------------+---------------------------+--------------------+------+--------------+-----+-----------------+--------------+--------------------+
|Love Cake|     Favorite Flavor|Other activities enjoyed|Perfect size of cake|     Cake experience|  Reason to buy Cake|Frequency of buying cake|When do you buy Cake?|   Place of Purchase|     Medium of order|Problem faced ordering cake|    Advise to bakers|Gender|     Residence|  Age|         category|     category1|    problem_category|
+---------+--------------------+------------------------+--------------------+--------------------+--------------------+------------------------+---------------------+--------------------+--------------------+---------------------------+--------------------+------+--------------+-----+-----------------+--------------

In [10]:

Top10 = spark.sql("SELECT `Favorite Flavor`, count(*) AS `Total no` FROM table1 survey GROUP BY `Favorite Flavor` ORDER BY `Total no` DESC")
Top10.show(10)

+---------------+--------+
|Favorite Flavor|Total no|
+---------------+--------+
|        Vanilla|       5|
|      Chocolate|       4|
|     Red velvet|       4|
|           null|       3|
|     Strawberry|       3|
|Chocolate fudge|       2|
|         Banana|       2|
|           Mint|       2|
|      Blueberry|       2|
|   Black Forest|       1|
+---------------+--------+
only showing top 10 rows



In [ ]:
Top10 = spark.sql("SELECT `Other activities enjoyed`, count(*) AS `Total no` FROM table1 survey GROUP BY `Other activities enjoyed` ORDER BY `Total no` DESC")
Top10.show(10)

+------------------------+--------+
|Other activities enjoyed|Total no|
+------------------------+--------+
|                    null|       2|
|                  Movies|       2|
|                 Reading|       2|
|         Baking ,cooking|       2|
|                 Cooking|       2|
|    Reading, writing,...|       1|
|             Researching|       1|
|                  Gaming|       1|
|                  Yogurt|       1|
|                Swimming|       1|
+------------------------+--------+
only showing top 10 rows



In [ ]:
Top5= spark.sql("SELECT `Perfect size of cake`, count(*) AS `Total no` FROM table1 survey GROUP BY `Perfect size of cake` ORDER BY `Total no` DESC")
Top5.show(5)

+--------------------+--------+
|Perfect size of cake|Total no|
+--------------------+--------+
|              Medium|      20|
|               Large|      10|
|               Small|       2|
|       Medium, Large|       2|
|                null|       1|
+--------------------+--------+
only showing top 5 rows



In [12]:
Top5 = spark.sql("""
    SELECT `Perfect size of cake`, count(*) AS `Total no`
    FROM table1 survey
    WHERE `Perfect size of cake` IS NOT NULL  -- Exclude rows with null values
    GROUP BY `Perfect size of cake`
    ORDER BY `Total no` DESC
""")
Top5.show(4)

+--------------------+--------+
|Perfect size of cake|Total no|
+--------------------+--------+
|              Medium|      20|
|               Large|      10|
|               Small|       2|
|       Medium, Large|       2|
+--------------------+--------+
only showing top 4 rows



In [ ]:
Top5= spark.sql("SELECT `Medium of order`, count(*) AS `Total no` FROM table1 survey GROUP BY `Medium of order` ORDER BY `Total no` DESC")
Top5.show(10)


+--------------------+--------+
|     Medium of order|Total no|
+--------------------+--------+
|Phone call, Whatsapp|      17|
|          Phone call|      12|
|            Whatsapp|       5|
|       All the above|       2|
|   Whatsapp, Twitter|       1|
|                null|       1|
|Phone call, Whats...|       1|
|               Email|       1|
+--------------------+--------+



In [ ]:
Top5= spark.sql("SELECT `Problem faced ordering cake`, count(*) AS `Total no` FROM table1 survey GROUP BY `Problem faced ordering cake` ORDER BY `Total no` DESC")
Top5.show(20)



+---------------------------+--------+
|Problem faced ordering cake|Total no|
+---------------------------+--------+
|                       null|       6|
|                       None|       3|
|                      Delay|       3|
|                None so far|       2|
|                     Delays|       2|
|       Delay in picking ...|       2|
|       Network and poor ...|       1|
|            Delayed replies|       1|
|       some bakeries wri...|       1|
|                    Colours|       1|
|       Confusion is n fl...|       1|
|       Sometimes they do...|       1|
|                        N/a|       1|
|       Delay especially ...|       1|
|        Haven't faced any..|       1|
|       Getting the order...|       1|
|             Late responses|       1|
|                        Non|       1|
|       Taking long and p...|       1|
|       Takes time to del...|       1|
+---------------------------+--------+
only showing top 20 rows



In [14]:
Top5= spark.sql("SELECT `AGE`, count(*) AS `Total no` FROM table1 survey WHERE AGE is not NULL GROUP BY `AGE` ORDER BY `Total no` DESC")
Top5.show(5)

+-----+--------+
|  AGE|Total no|
+-----+--------+
|27-30|      12|
|24-26|      12|
|18-23|      12|
|  30<|       3|
+-----+--------+



In [ ]:
Top3= spark.sql("SELECT `Residence`, count(*) AS `Total no` FROM table1 survey GROUP BY `Residence` ORDER BY `Total no` DESC")
Top3.show(23)

+----------------+--------+
|       Residence|Total no|
+----------------+--------+
|         Kampala|       9|
|            null|       6|
|       Namugongo|       3|
|            Kira|       2|
|          Ntinda|       2|
|          Gayaza|       2|
|     Kisosonkole|       1|
|        Personal|       1|
|          Kasubi|       1|
|        Makerere|       1|
|  Lira currently|       1|
|              US|       1|
|  Buziga kampala|       1|
|            Lira|       1|
|          Nalyaa|       1|
|    Kyaliwajjala|       1|
|        Kyambogo|       1|
|        Munyonyo|       1|
|            Gulu|       1|
|         Muyenga|       1|
|Entebbe and Gulu|       1|
|         Bulenga|       1|
+----------------+--------+



In [13]:
Top3= spark.sql("SELECT `Gender`, count(*) AS `Total no` FROM table1 survey WHERE Gender is not NULL GROUP BY `Gender` ORDER BY `Total no` DESC")
Top3.show(3)

+------+--------+
|Gender|Total no|
+------+--------+
|Female|      27|
|  Male|      12|
+------+--------+

